In [ ]:
# Example Notebook for DeepLabCut(DLC) manual ingestion steps
from workflow.pipeline.dlc import insert_new_dlc_model
from workflow.pipeline import reference, train, model
from pathlib import Path

In [2]:
# Instruction for ingesting new models into the model.Model() table

# Set project path to specify the location of the config.yaml file
# This is the only required paramter
example_project_path = "/home/anaconda/inbox/dlc_projects/Testing2D-Kim-2022-01-04"

# None of the following need to be specified, all are optional parameters
model_prefix = "dlc_test_"
model_description = "Test Model Ingestion for DLC"
paramset_idx = 1

insert_new_dlc_model(project_path=example_project_path, 
                    model_prefix=model_prefix, 
                    paramset_idx=paramset_idx, 
                    model_description=model_description)
        
# Display model.Model() table to see insert
model.Model()

In [ ]:
# Manual step to insert Camera(s) into reference.Device()
# This step is required for the model.VideoRecording table to be populated

# For example 
device_id=0,
device_name='camera1', 
device_description='dlc test camera'
reference_dict = dict(device_id=device_id,
                    device_name=device_name, 
                    device_description=device_description)
reference.Device.insert1(reference_dict)
reference.Device()

In [ ]:
# Manual Step to insert entry into Pose Estimation Task

# Choose a video recording by manually specifying recording
# pe_key = (model.VideoRecording & {'recording_id': '0'}).fetch1('KEY')
# Or another method
vr_key = dict(subject='Subject1',
            session_id=1,
            recording_id=0)
pe_key_1 = (model.VideoRecording & vr_key).fetch1('KEY')
# Specify Model to be used for Pose Estimation Inference
model_key = (model.Model & 'model_name LIKE "%Testing2D-Kim%"').fetch1('KEY')

pe_key_1.update(model_key)
# pe_key.update(model_key)

task_mode = "trigger" # Can be set to trigger (to trigger inference) 
                      # or load (to load external results into database)

#  Params Optional(default None). Parameters passed to DLC's analyze_videos:
#                 videotype, gputouse, save_as_csv, batchsize, cropping, TFGPUinference,
#                 dynamic, robust_nframes, allow_growth, use_shelve
params={'save_as_csv':True}

# Use built in infer_output_dir function to generate output directory location
# Or can manually specify this  
output_dir = model.PoseEstimationTask.infer_output_dir(pe_key_1)
skip_duplicates = False # Manually set whether duplicate entries should be skipped or retried

model.PoseEstimationTask.insert_estimation_task(
        key=pe_key_1,
        task_mode=task_mode,
        params=params,
        relative=True,
        mkdir=True,
        skip_duplicates=False,
    )

model.PoseEstimationTask.insert1(
            {
                **pe_key_1,
                "task_mode": task_mode,
                "pose_estimation_params": params,
                "pose_estimation_output_dir": output_dir,
            },
            skip_duplicates=skip_duplicates,
        )



In [ ]:
# Manually insert Training Video Set ID (increment for each training set)
train.VideoSet.insert1({'video_set_id': 0})

# Manually set project path
relative_project_path = "dlc_projects/Testing2D-Kim-2022-01-04/videos"

# Training videos should be stored within the project directory in subfolders for videos and labeled data
full_videos_project_path = Path(model.get_dlc_root_data_dir()[0]) / 'dlc_projects' / 'Testing2D-Kim-2022-01-04' / 'videos'
full_labeled_project_path = Path(model.get_dlc_root_data_dir()[0]) / 'dlc_projects' / 'Testing2D-Kim-2022-01-04' / 'labeled-data'

# Video and labeled data files are auto parsed and paths are stored
example_video_files = [tfile for tfile in full_videos_project_path.glob('*') if tfile.is_file()]
example_labeled_csv_files = [lfile for lfile in full_labeled_project_path.rglob('*.csv') if lfile.is_file()]
example_labeled_png_files = [pfile for pfile in full_labeled_project_path.rglob('*.png') if pfile.is_file()]
example_labeled_h5_files = [hfile for hfile in full_labeled_project_path.rglob('*.h5') if hfile.is_file()]
example_training_files = example_video_files + example_labeled_csv_files + example_labeled_png_files + example_labeled_h5_files
project_folder = relative_project_path
# Insert all files associated with that training set
for idx, filename in enumerate(example_training_files):
    train.VideoSet.File.insert1({'video_set_id': 0,
                                 'file_id': idx,
                                 'file_path': (str(project_folder) + str(filename))})

train.VideoSet.File()


In [ ]:
# Manually insert Training Parameter Set
import yaml

paramset_idx = 0; 
paramset_desc='dlc project description: Testing2D-Kim'
example_project_path = "/home/anaconda/inbox/dlc_projects/Testing2D-Kim-2022-01-04"
config_path = example_project_path / 'config.yaml'

with open(config_path, 'rb') as y:
    config_params = yaml.safe_load(y)
example_training_params = {'shuffle': '1',
                   'trainingsetindex': '0',
                   'maxiters': '5',
                   'scorer_legacy': 'False',
                   'maxiters': '5', 
                   'multianimalproject':'False'}
config_params.update(example_training_params)
train.TrainingParamSet.insert_new_params(paramset_idx=paramset_idx,
                                         paramset_desc=paramset_desc,
                                         params=config_params)

train.TrainingParamSet()
